# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921701825916                   -0.69    5.6         
  2   -7.926164652821       -2.35       -1.22    1.0    248ms
  3   -7.926836725298       -3.17       -2.37    1.8    275ms
  4   -7.926861490527       -4.61       -3.03    2.9    342ms
  5   -7.926861647824       -6.80       -3.41    1.9    257ms
  6   -7.926861671086       -7.63       -3.83    1.5    242ms
  7   -7.926861680305       -8.04       -4.30    1.2    236ms
  8   -7.926861681756       -8.84       -4.86    1.9    255ms
  9   -7.926861681854      -10.01       -5.16    2.0    302ms
 10   -7.926861681870      -10.79       -5.70    1.6    251ms
 11   -7.926861681872      -11.61       -6.39    1.9    261ms
 12   -7.926861681873      -12.89       -7.75    2.1    273ms
 13   -7.926861681873      -14.45       -7.79    4.6    378ms
 14   -7.926861681873      -15.05       -8.10    1.0    234ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921670082805                   -0.69    5.9         
  2   -7.926158858249       -2.35       -1.22    1.0    248ms
  3   -7.926837155361       -3.17       -2.37    1.8    273ms
  4   -7.926861488476       -4.61       -3.05    2.8    298ms
  5   -7.926861652254       -6.79       -3.44    1.9    322ms
  6   -7.926861672247       -7.70       -3.86    1.4    240ms
  7   -7.926861680147       -8.10       -4.26    1.4    239ms
  8   -7.926861681752       -8.79       -4.87    1.6    249ms
  9   -7.926861681854       -9.99       -5.16    1.8    257ms
 10   -7.926861681870      -10.78       -5.74    1.6    289ms
 11   -7.926861681873      -11.63       -6.66    1.6    255ms
 12   -7.926861681873      -13.07       -7.33    2.9    308ms
 13   -7.926861681873      -14.45       -7.66    2.5    276ms
 14   -7.926861681873   +  -14.57       -8.72    1.4    262ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921687525138                   -0.69    5.9         
  2   -7.926169389420       -2.35       -1.22    1.0    255ms
  3   -7.926841088206       -3.17       -2.37    1.8    294ms
  4   -7.926864935444       -4.62       -3.03    2.8    322ms
  5   -7.926865050364       -6.94       -3.38    1.6    256ms
  6   -7.926865079570       -7.53       -3.76    1.9    267ms
  7   -7.926865091634       -7.92       -4.36    1.1    280ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
